In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.linalg import lstsq
import scipy.linalg as linalg 
from scipy.linalg import solve
from scipy.stats import norm
import math

# LSMC

In [ ]:
def american_lsmc(S0, K, T, r, sigma, poly_type, k, nsim=100_000):
    # setting in question, change accordingly
    dt = 1/np.sqrt(nsim)
    n = int(T // dt)
    discount = np.exp(-r * dt)
    
    # Antithtic variates
    Z = np.random.randn(nsim // 2, n + 1)
    Z = np.vstack((Z, -Z))
    stock = np.zeros((nsim, n + 1))
    stock[:, 0] = S0
    
    # Simulate stock prices
    for i in range(n):
        stock[:, i + 1] = stock[:, i] + stock[:, i] * r * dt + sigma * np.sqrt(dt) * stock[:, i] * Z[:, i]
    
    # Simulated stock price
    stock = stock[:, 1:]
    # payoff without early exercise
    EV = np.maximum(K - stock, 0)


    Idx = np.zeros((nsim, n), dtype=bool)
    #initialize the index at maturity
    Idx[:, -1] = EV[:, -1] > 0
    
    for i in range(n - 2, -1, -1):
        inMoney = np.where(EV[:, i] > 0)[0]
        X = stock[inMoney, i]
        Y = np.sum(discount * Idx * EV, axis=1)[inMoney]
        
        EV[:, i + 1:] = discount * EV[:, i + 1:]
        # calculated use different method
        ecv = ECV(X, Y, poly_type, k)

        Idx[inMoney, i] = EV[inMoney, i] > ecv
        Idx[np.cumsum(Idx, axis=1) > 1] = 0

    EV = np.maximum(K - stock, 0)
    discount_matrix = np.exp(-r * np.arange(1, n + 1) * dt).reshape(1, -1)
    price = np.mean(np.sum(discount_matrix * Idx * EV, axis=1))
    return price

def ECV(X, Y, poly_type, k):
    n = len(X)
    
    if poly_type == 'hermite':
        f1 = np.ones(n)
        f2 = 2 * X
        if k == 2:
            f = np.column_stack((f1, f2))
        elif k == 3:
            f3 = 4 * X**2 - 2
            f = np.column_stack((f1, f2, f3))
        elif k == 4:
            f3 = 4 * X**2 - 2
            f4 = 8 * X**3 - 12 * X
            f = np.column_stack((f1, f2, f3, f4))
    
    elif poly_type == 'monomial':
        f1 = np.ones(n)
        f2 = X
        if k == 2:
            f = np.column_stack((f1, f2))
        elif k == 3:
            f3 = X**2
            f = np.column_stack((f1, f2, f3))
        elif k == 4:
            f3 = X**2
            f4 = X**3
            f = np.column_stack((f1, f2, f3, f4))
    
    elif poly_type == 'laguerre':
        f1 = np.ones(n)
        f2 = f1 * (1 - X)
        if k == 2:
            f = np.column_stack((f1, f2))
        elif k == 3:
            f3 = f1 * (1 - 2 * X + X**2 / 2)
            f = np.column_stack((f1, f2, f3))
        elif k == 4:
            f3 = f1 * (1 - 2 * X + X**2 / 2)
            f4 = f1 * (1 - 3 * X + 3 * X**2 / 2 - X**3 / 6)
            f = np.column_stack((f1, f2, f3, f4))
    
    A = f.T @ f
    b = f.T @ Y
    a = np.linalg.lstsq(A, b, rcond=None)[0]
    res = np.sum(a * f, axis=1)
    
    return res

# Example usage
S0 = 180
K = 180
T = 0.5
r = 0.055
sigma = 0.25
poly_type = 'laguerre'
k = 3

price = american_lsmc(S0, K, T, r, sigma, poly_type, k)
print("American option price:", price)
